# Рекурентные сети для обработки последовательностей

Вспомним все, что мы уже знаем про обработку текстов:
- Компьютер не понимает текст, поэтому нам нужно его как-то закодировать - представить в виде вектора или эмбеддинга
- Слово - это токен
- В тексте много повторяющихся слов/лишний слов - нужно сделать препроцессинг:
    - удалить знаки препинания
    - удалить стоп-слова
    - Удаляем html-теги, схлопываем текст, удаляем спецсимволы
    - привести слова к начальной форме (**стемминг** и **лемматизация**)
   
   Лемматизация - заменяет грамматическое окончание суффиксом или окончанием начальной формы
   
    
- После этого мы можем представить наш текст (набор слов) в виде вектора, например, стандартными способами:
    - **CounterEncoding** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=1,\ если\ vocab[i]\ \in\ doc$
    - **TfIdfVectorizer** - вектор длины размер нашего словаря
        - есть словарь vocab, который можем включать слова, ngram-ы
        - каждому документу $doc$ ставим в соответствие вектор $vec\ :\ vec[i]=tf(vocab[i])*idf(vocab[i]),\ если\ vocab[i]\ \in\ doc$
    
        $$ tf(t,\ d)\ =\ \frac{n_t}{\sum_kn_k} $$
        $$ idf(t,\ D)\ =\ \log\frac{|D|}{|\{d_i\ \in\ D|t\ \in\ D\}|} $$



* Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.


* TF (term frequency — частота слова) — отношение числа вхождений некоторого слова к общему числу слов документа. Таким образом, оценивается важность слова в пределах отдельного документа.

* IDF (inverse document frequency — обратная частота документа) — инверсия частоты, с которой некоторое слово встречается в документах коллекции. 

* Большой вес в TF-IDF получат слова с высокой частотой в пределах конкретного документа и с низкой частотой употреблений в других документах.

, где 
- $n_t$ - число вхождений слова $t$ в документ, а в знаменателе — общее число слов в данном документе
- $|D|$ — число документов в коллекции;
- $|\{d_i\ \in\ D\mid\ t\in d_i\}|$— число документов из коллекции $D$, в которых встречается $t$ (когда $n_t\ \neq\ 0$).



Это база и она работает. Мы изучили более продвинутые подходы: эмбединги и сверточные сети по эмбедингам. Но тут есть проблема: любой текст - это последовательность, ни эмбединги, ни сверточные сети не работают с ним как с последовательностью. Так давайте попробуем придумать архитектуру, которая будет работать с текстом как с последовательностью, двигаясь по эмбедингам и как-то меняя их значения.

In [1]:
import nltk
#nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("russian")
stopwords = list(set(stopwords))
print(' '.join([x for x in stopwords[:20]]))
print('*'*100)


tokens = 'Князь равнодушно замолк, Анна Павловна, с свойственною ей придворною и женскою ловкостью и быстротою такта, захотела и щелкануть князя за то, что он дерзнул так отозваться о лице, рекомендованном императрице, и в то же время утешить его.'.split()
tokens = [str(x).lower() for x in tokens]


stemmer = nltk.stem.SnowballStemmer('russian')
stemmed = []
for token in tokens:
    if token != stemmer.stem(token):
        stemmed.append(token + "-" + stemmer.stem(token))
print(stemmed)
print('*'*100)


#pymorphy2
# - приводить слово к нормальной форме (например, “люди -> человек”, или “гулял -> гулять”).
# - ставить слово в нужную форму. Например, ставить слово во множественное число, менять падеж слова и т.д.
# - возвращать грамматическую информацию о слове (число, род, падеж, часть речи и т.д.)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
lemm = []
for token in tokens:
    if token != morph.parse(token)[0][2]:
        lemm.append(token + "-" + morph.parse(token)[0][2])
print(lemm)
print('*'*100)

нельзя бы тут никогда наконец еще и чтоб него чего есть раз этот зачем моя там вот между потому я
****************************************************************************************************
['князь-княз', 'равнодушно-равнодушн', 'анна-ан', 'свойственною-свойствен', 'ей-е', 'придворною-придворн', 'женскою-женск', 'ловкостью-ловкост', 'быстротою-быстрот', 'захотела-захотел', 'щелкануть-щелканут', 'князя-княз', 'отозваться-отозва', 'рекомендованном-рекомендова', 'время-врем', 'утешить-утеш']
****************************************************************************************************
['свойственною-свойственный', 'ей-она', 'придворною-придворный', 'женскою-женский', 'ловкостью-ловкость', 'быстротою-быстрота', 'захотела-захотеть', 'князя-князь', 'дерзнул-дерзнуть', 'рекомендованном-рекомендовать']
****************************************************************************************************


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

a = 'Князь равнодушно замолк, Анна Павловна, с свойственною ей придворною и женскою ловкостью и быстротою такта, захотела и щелкануть князя за то, что он дерзнул так отозваться о лице, рекомендованном императрице, и в то же время утешить его.'
b = '— Я часто думаю, — продолжала Анна Павловна после минутного молчания, придвигаясь к князю и ласково улыбаясь ему, как будто выказывая этим, что политические и светские разговоры кончены и теперь начинается задушевный, — я часто думаю, как иногда несправедливо распределяется счастие жизни.'
c = 'Приехала высшая знать Петербурга, люди самые разнородные по возрастам и характерам, но одинаковые по обществу, в каком все жили; приехала дочь князя Василия, красавица Элен, заехавшая за отцом, чтобы с ним вместе ехать на праздник посланника.'
df = pd.DataFrame.from_dict({0:a, 1:b, 2:c}, orient='index').rename(columns={'index':'event', 0:'text'})

def lemma(txt):
    return ' '.join([morph.parse(x)[0].normal_form for x in txt.split()])

df['text'] = df['text'].apply(lambda x: str(x).replace('—', '').replace(',', '').replace('.', ''))
df['text'] = df['text'].apply(lambda x: lemma(x))

#vectorizer=TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1, 2), min_df=0.0005, max_df=0.995, max_features = None)
vectorizer=TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1, 2), min_df=0.0, max_df=1.0, max_features = None)
X = vectorizer.fit_transform(df['text'])

count_vect_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df = pd.concat([df, count_vect_df], axis=1)

In [3]:
df

,text,анна,анна павлович,будто,будто выказывать,быстрота,быстрота такт,василий,василий красавица,вместе,...,что он,что политический,чтобы,чтобы они,щелкануть,щелкануть князь,элен,элен заехать,это,это что
0,князь равнодушно замолкнуть анна павлович с св...,0.099349,0.099349,0.000000,0.000000,0.130631,0.130631,0.000000,0.000000,0.000000,...,0.130631,0.000000,0.000000,0.000000,0.130631,0.130631,0.000000,0.000000,0.000000,0.000000
1,я часто думать продолжать анна павлович после ...,0.090744,0.090744,0.119318,0.119318,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.119318,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119318,0.119318
2,приехать высокий знать петербург человек самый...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.121333,0.121333,0.121333,...,0.000000,0.000000,0.121333,0.121333,0.000000,0.000000,0.121333,0.121333,0.000000,0.000000


In [4]:
df[[x for x in df.columns if x.count('анна') > 0 or x.count('князь') > 0]]

,анна,анна павлович,дочь князь,замолкнуть анна,князь,князь василий,князь за,князь ласково,князь равнодушно,придвигаться князь,продолжать анна,щелкануть князь
0,0.099349,0.099349,0.000000,0.130631,0.154306,0.000000,0.130631,0.000000,0.130631,0.000000,0.000000,0.130631
1,0.090744,0.090744,0.000000,0.000000,0.070471,0.000000,0.000000,0.119318,0.000000,0.119318,0.119318,0.000000
2,0.000000,0.000000,0.121333,0.000000,0.071661,0.121333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [5]:
#df.columns.to_list()

Если документ содержит 100 слов, и слово[3] «заяц» встречается в нём 3 раза, то частота слова (TF) для слова «заяц» в документе будет 0,03 (3/100). Вычислим IDF как десятичный логарифм отношения количества всех документов к количеству документов, содержащих слово «заяц». Таким образом, если «заяц» содержится в 1000 документах из 10 000 000 документов, то IDF будет равной: log(10 000 000/1000) = 4. Для расчета окончательного значения веса слова необходимо TF умножить на IDF. В данном примере, TF-IDF вес для слова «заяц» в выбранном документе будет равен: 0,03 × 4 = 0,12.

In [6]:
df = pd.DataFrame.from_dict({0:a.lower(), 1:b.lower(), 2:c.lower()}, orient='index').rename(columns={'index':'event', 0:'text'})
df['text'] = df['text'].apply(lambda x: str(x).replace('—', '').replace(',', '').replace('.', ''))
df['text'] = df['text'].apply(lambda x: lemma(x))
vectorizer=TfidfVectorizer(analyzer='word', lowercase=False, ngram_range=(1, 1), min_df=0.0, max_df=1.0, max_features=None)#, sublinear_tf=True , norm=None, use_idf=True, sublinear_tf=False, binary=True)
X = vectorizer.fit_transform(df['text'])
count_vect_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names())
df = pd.concat([df, count_vect_df], axis=1)

In [7]:
wd = "анна"
sm = 0
tf = []
for i in range(3):
    tf.append(df.loc[i,"text"].count(wd)/len(df.loc[i,"text"].split()))
    print(f'{i}. ngram={len(df.loc[i,"text"].split())}   TF={df.loc[i,"text"].count(wd)/len(df.loc[i,"text"].split())}')
    sm += len(df.loc[i,"text"].split())
sm

0. ngram=37   TF=0.02702702702702703
1. ngram=39   TF=0.02564102564102564
2. ngram=36   TF=0.0


112

In [8]:
v0 = np.sum([df.loc[i,"text"].count(wd) for i in range(len(df))])
v1 = len(df)
v2 = np.log10(v1/v0)
print(v0, v1, v2)

2 3 0.17609125905568124


In [9]:
[x*v2 for x in tf]

[0.0047592232177211145, 0.004515160488607211, 0.0]

In [10]:
df[[x for x in df.columns if x.count('анна') > 0 or x.count('князь') > 0]]

,анна,князь
0,0.138800,0.215582
1,0.125604,0.097543
2,0.000000,0.098536
